In [2]:
pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time
import pandas as pd
import time
from bs4 import BeautifulSoup


options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Запуск в headless-режиме
options.add_argument("--no-sandbox")  # Отключение sandbox
options.add_argument("--disable-dev-shm-usage")  # Решение для ограниченной памяти
options.add_argument("--disable-gpu")  # Отключение GPU
options.add_argument("--remote-debugging-port=9222")
driver = webdriver.Chrome(options=options)

In [4]:
#first check
try:
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.inmotionhosting.com/")
    print(f"Title: {driver.title}")
except Exception as e:
    print(f"Ошибка: {e}")
finally:
    driver.quit()



Title: InMotion Hosting | Premier Web Host & Server Provider


In [5]:
driver = webdriver.Chrome(options=options)
driver.get("https://www.inmotionhosting.com/")

###Fighting with cookie baner
try:

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "onetrust-banner-sdk")))
    print("Cookie banner detected.")


    cookie_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
    cookie_button.click()
    print("Cookie banner accepted.")
    WebDriverWait(driver, 10).until(
        EC.invisibility_of_element((By.ID, "onetrust-banner-sdk")))
    print("Cookie banner is no longer visible.")
except Exception as e:
    print(f"Cookie banner handling error: {e}")

Cookie banner handling error: Message: 
Stacktrace:
#0 0x5b76aab088fa <unknown>
#1 0x5b76aa619d20 <unknown>
#2 0x5b76aa668a66 <unknown>
#3 0x5b76aa668d01 <unknown>
#4 0x5b76aa6ae184 <unknown>
#5 0x5b76aa68cb1d <unknown>
#6 0x5b76aa6ab560 <unknown>
#7 0x5b76aa68c893 <unknown>
#8 0x5b76aa65b30d <unknown>
#9 0x5b76aa65c32e <unknown>
#10 0x5b76aaad500b <unknown>
#11 0x5b76aaad8f97 <unknown>
#12 0x5b76aaac171c <unknown>
#13 0x5b76aaad9b17 <unknown>
#14 0x5b76aaaa66cf <unknown>
#15 0x5b76aaaf76b8 <unknown>
#16 0x5b76aaaf7880 <unknown>
#17 0x5b76aab07776 <unknown>
#18 0x7af7d6725ac3 <unknown>



In [6]:
# go to page with all plans
try:
    target_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#home-rostrum-3 > div > div > div:nth-child(1) > div.imh-pricing-container.connected-switcher > div.active.imh-switcher > a")))
    driver.execute_script("arguments[0].scrollIntoView();", target_button)
    driver.execute_script("arguments[0].click();", target_button)
    print("Target button clicked successfully via JavaScript!")

except Exception as e:
    print(f"Error while clicking the target button: {e}")

try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#shared-hosting-spec-table > div > div.imh-spec-table-container")))
    print("Table container loaded!")
except Exception as e:
    print(f"Error loading table container: {e}")

html_content = driver.page_source #getting a html and parsing

soup = BeautifulSoup(html_content, "html.parser")

#searching the table container and table
table_container = soup.select_one("#shared-hosting-spec-table > div > div.imh-spec-table-container")

table = table_container.select_one("table")

# take a headers
headers = [header.text.strip() for header in table.find("thead").find_all("th")]

# take the rows
rows = []
for row in table.find("tbody").find_all("tr"):
    cells = [cell.text.strip() for cell in row.find_all("td")]
    rows.append(cells)


df_1year = pd.DataFrame(rows, columns=headers)


df_1year.to_csv("1_year_host_plans.csv", index=False)


print("Data scraped and saved successfully!")

Target button clicked successfully via JavaScript!
Table container loaded!
Data scraped and saved successfully!


In [7]:
#separate parcing (price)
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#shared-hosting-spec-table > div > div.imh-spec-table-container > table > thead.desktop-view")))
    print("Found the target element!")
except Exception as e:
    print(f"Error finding the element: {e}")
    driver.quit()


html_content = element.get_attribute("outerHTML")


soup = BeautifulSoup(html_content, "html.parser")


columns = soup.select("th")

price_data = []

for column in columns:
    try:
        # name of the plan
        plan_name = column.select_one("span.column-title").text.strip()

        # price
        price_block = column.select_one("div.imh-switcher.active div.pricing-container span.rostrum-price")
        price = price_block.text.strip() if price_block else "N/A"

        # savings
        savings_block = column.select_one("div.imh-switcher.active div.imh-save-ribbon span.dnt")
        savings = savings_block.text.strip() if savings_block else "N/A"

        #
        price_data.append({
            "Plan": plan_name,
            "Price (1 Year)": price,
            "Savings (1 Year)": savings
        })
    except Exception as e:
        print(f"Error processing column: {e}")


df_1year_price = pd.DataFrame(price_data)


df_1year_price.to_csv("1year_plan_price.csv", index=False)

print("Data scraped successfully!")
print(df_1year_price)

Found the target element!
Error processing column: 'NoneType' object has no attribute 'text'
Data scraped successfully!
     Plan Price (1 Year) Savings (1 Year)
0    Core          $2.99              73%
1  Launch          $4.79              66%
2   Power          $4.79              73%
3     Pro         $10.79              57%


In [26]:
#Searching and clicking the button with other plans

try:
    button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//button[contains(text(), '3 Year')]"))
    )
    print(f"Found button: {button.text}")
except Exception as e:
    print(f"Error finding the '3 Year' button: {e}")


try:
    driver.execute_script("arguments[0].click();", button)
    print("Clicked on '3 Year' button.")
except Exception as e:
    print(f"Error clicking on the '3 Year' button: {e}")


Found button: 3 Year
Clicked on '3 Year' button.


In [27]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#shared-hosting-spec-table > div > div.imh-spec-table-container > table > thead.desktop-view"))
    )
    print("Found the target element!")
except Exception as e:
    print(f"Error finding the element: {e}")
    driver.quit()


html_content = element.get_attribute("outerHTML")


soup = BeautifulSoup(html_content, "html.parser")


columns = soup.select("th")

price_data_3year = []

for column in columns:
    try:
        # name of the plan
        plan_name = column.select_one("span.column-title").text.strip()

        # price
        price_block = column.select_one("div.imh-switcher.active div.pricing-container span.rostrum-price")
        price = price_block.text.strip() if price_block else "N/A"

        # savings
        savings_block = column.select_one("div.imh-switcher.active div.imh-save-ribbon span.dnt")
        savings = savings_block.text.strip() if savings_block else "N/A"

        #
        price_data_3year.append({
            "Plan": plan_name,
            "Price (3 Year)": price,
            "Savings (3 Year)": savings
        })
    except Exception as e:
        print(f"Error processing column: {e}")

# Создаем DataFrame
df_3year_price = pd.DataFrame(price_data_3year)

# Сохраняем в CSV
df_3year_price.to_csv("3year_plan_price.csv", index=False)

print("Data scraped successfully!")
print(df_3year_price)

Found the target element!
Error processing column: 'NoneType' object has no attribute 'text'
Data scraped successfully!
     Plan Price (3 Year) Savings (3 Year)
0    Core          $3.19              68%
1  Launch          $4.99              62%
2   Power          $4.99              71%
3     Pro         $10.99              54%


In [10]:

try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#shared-hosting-spec-table > div > div.imh-spec-table-container"))
    )
    print("Table container loaded!")
except Exception as e:
    print(f"Error loading table container: {e}")

html_content = driver.page_source #getting a html and parsing

soup = BeautifulSoup(html_content, "html.parser")

#searching the table container and table
table_container = soup.select_one("#shared-hosting-spec-table > div > div.imh-spec-table-container")

table = table_container.select_one("table")

# take a headers
headers = [header.text.strip() for header in table.find("thead").find_all("th")]

# take the rows
rows = []
for row in table.find("tbody").find_all("tr"):
    cells = [cell.text.strip() for cell in row.find_all("td")]
    rows.append(cells)


df_3year = pd.DataFrame(rows, columns=headers)


df_3year.to_csv("3_years_host_plans.csv", index=False)


print("Data scraped and saved successfully!")

Table container loaded!
Data scraped and saved successfully!


In [28]:
# Searching and clicking button for short plans (1 month)
try:
    button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//button[contains(text(), '1 Month')]"))
    )
    print(f"Found button: {button.text}")
except Exception as e:
    print(f"Error finding the '1 Month' button: {e}")

try:
    driver.execute_script("arguments[0].click();", button)
    print("Clicked on '1 Month' button.")
except Exception as e:
    print(f"Error clicking on the '1 Month' button: {e}")

Found button: 1 Month
Clicked on '1 Month' button.


In [29]:
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#shared-hosting-spec-table > div > div.imh-spec-table-container > table > thead.desktop-view"))
    )
    print("Found the target element!")
except Exception as e:
    print(f"Error finding the element: {e}")
    driver.quit()


html_content = element.get_attribute("outerHTML")


soup = BeautifulSoup(html_content, "html.parser")


columns = soup.select("th")

price_data_1month = []

for column in columns:
    try:
        # name of the plan
        plan_name = column.select_one("span.column-title").text.strip()

        # price
        price_block = column.select_one("div.imh-switcher.active div.pricing-container span.rostrum-price")
        price = price_block.text.strip() if price_block else "N/A"

        # savings
        savings_block = column.select_one("div.imh-switcher.active div.imh-save-ribbon span.dnt")
        savings = savings_block.text.strip() if savings_block else "N/A"

        #
        price_data_1month.append({
            "Plan": plan_name,
            "Price (1 Month)": price,
            "Savings (1 Month)": savings
        })
    except Exception as e:
        print(f"Error processing column: {e}")

# Создаем DataFrame
df_1month_price = pd.DataFrame(price_data_1month)

# Сохраняем в CSV
df_3year_price.to_csv("1month_plan_price.csv", index=False)

print("Data scraped successfully!")
print(df_1month_price)

Found the target element!
Error processing column: 'NoneType' object has no attribute 'text'
Data scraped successfully!
     Plan Price (1 Month) Savings (1 Month)
0    Core             N/A               N/A
1  Launch          $15.99                0%
2   Power          $15.99               20%
3     Pro          $19.99               26%


In [13]:
#repeat third time the parcing

try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#shared-hosting-spec-table > div > div.imh-spec-table-container"))
    )
    print("Table container loaded!")
except Exception as e:
    print(f"Error loading table container: {e}")

html_content = driver.page_source #getting a html and parsing

soup = BeautifulSoup(html_content, "html.parser")

#searching the table container and table
table_container = soup.select_one("#shared-hosting-spec-table > div > div.imh-spec-table-container")

table = table_container.select_one("table")

# take a headers
headers = [header.text.strip() for header in table.find("thead").find_all("th")]

# take the rows
rows = []
for row in table.find("tbody").find_all("tr"):
    cells = [cell.text.strip() for cell in row.find_all("td")]
    rows.append(cells)


df_1month = pd.DataFrame(rows, columns=headers)


df_1month.to_csv("1month_host_plans.csv", index=False)


print("Data scraped and saved successfully!")

Table container loaded!
Data scraped and saved successfully!


In [14]:
df_1month_copy = df_1month.copy()
df_3year_copy = df_3year.copy()
df_1year_copy = df_1year.copy()

In [15]:
df_1month = df_1month.drop(index=0)
df_3year = df_3year.drop(index=0)
df_1year = df_1year.drop(index=0)

In [36]:
df_1month

,Features,Core 1 month,Launch 1 month,Power 1 month,Pro 1 month
1,Websites Supported,2 Websites,25 Websites,Unlimited Websites,Unlimited Websites
2,Disk Space,100 GB SSD,Unlimited NVMe SSD,Unlimited NVMe SSD,Unlimited NVMe SSD
3,Bandwidth,Unlimited,Unlimited,Unlimited,Unlimited
4,Email Accounts,10 Email Addresses,Unlimited Email Addresses,Unlimited Email Addresses,Unlimited Email Addresses
5,Dedicated IP Address,Available,Available,Available,
6,No-Downtime Website Migration,Available,Available,Available,Available
7,UltraStack Optimized Performance,2x Speed and Performance,6X UltraStack Speed & Performance,12X UltraStack Speed & Performance,20X UltraStack Speed & Performance
8,Free SSL,,,,
9,eCommerce Ready,,,,
10,Pro Level Support,,,,


In [17]:
df_1month.columns.values[0] = 'Features'
df_3year.columns.values[0] = 'Features'
df_1year.columns.values[0] = 'Features'

In [18]:
df_1month.columns.values[1] = 'Core 1 month'
df_3year.columns.values[1] = 'Core 3 Years'
df_1year.columns.values[1] = 'Core 1 Year'



In [19]:
df_1month.columns.values[2] = 'Launch 1 month'
df_3year.columns.values[2] = 'Launch 3 Years'
df_1year.columns.values[2] = 'Launch 1 Year'

In [20]:
df_1month.columns.values[3] = 'Power 1 month'
df_3year.columns.values[3] = 'Power 3 Years'
df_1year.columns.values[3] = 'Power 1 Year'

In [21]:
df_1month.columns.values[4] = 'Pro 1 month'
df_3year.columns.values[4] = 'Pro 3 Years'
df_1year.columns.values[4] = 'Pro 1 Year'

In [22]:
df_1month # look nice!

,Features,Core 1 month,Launch 1 month,Power 1 month,Pro 1 month
1,Websites Supported,2 Websites,25 Websites,Unlimited Websites,Unlimited Websites
2,Disk Space,100 GB SSD,Unlimited NVMe SSD,Unlimited NVMe SSD,Unlimited NVMe SSD
3,Bandwidth,Unlimited,Unlimited,Unlimited,Unlimited
4,Email Accounts,10 Email Addresses,Unlimited Email Addresses,Unlimited Email Addresses,Unlimited Email Addresses
5,Dedicated IP Address,Available,Available,Available,
6,No-Downtime Website Migration,Available,Available,Available,Available
7,UltraStack Optimized Performance,2x Speed and Performance,6X UltraStack Speed & Performance,12X UltraStack Speed & Performance,20X UltraStack Speed & Performance
8,Free SSL,,,,
9,eCommerce Ready,,,,
10,Pro Level Support,,,,


In [23]:
df_1year_price

,Plan,Price (1 Year),Savings (1 Year)
0,Core,$2.99,73%
1,Launch,$4.79,66%
2,Power,$4.79,73%
3,Pro,$10.79,57%


In [30]:
df_price_and_savings = pd.concat([df_1year_price, df_3year_price, df_1month_price], axis=1)

In [35]:
df_price_and_savings

,Plan,Price (1 Year),Savings (1 Year),Price (3 Year),Savings (3 Year),Price (1 Month),Savings (1 Month)
0,Core,$2.99,73%,$3.19,68%,N/A,N/A
1,Launch,$4.79,66%,$4.99,62%,$15.99,0%
2,Power,$4.79,73%,$4.99,71%,$15.99,20%
3,Pro,$10.79,57%,$10.99,54%,$19.99,26%


In [34]:
df_price_and_savings = df_price_and_savings.loc[:, ~df_price_and_savings.columns.duplicated()]

In [42]:
df_price_and_savings.to_csv("price_and_savings.csv", index=False)

In [37]:
df_features_and_plans = pd.concat([df_1year, df_3year, df_1month], axis=1)

In [39]:
df_features_and_plans = df_features_and_plans.loc[:, ~df_features_and_plans.columns.duplicated()]

In [43]:
df_features_and_plans.to_csv("features_and_plans.csv", index=False)

I haven't figured out how to parse the ticks and cons from the table